In [15]:
# HTMLの取得
import requests
from bs4 import BeautifulSoup
import time
import re #スター数の表記を変換するために使用
import sqlite3

In [16]:
# データベース設定
path = ''
db_name = 'github_repositories.db'

def create_database():
    """データベースとテーブルを作成"""
    try:
        # DB接続オブジェクトの作成
        conn = sqlite3.connect(path + db_name)
        
        # SQLを実行するためのカーソルオブジェクトを取得
        cur = conn.cursor()
        
        # SQL文の作成 - テーブルの作成
        sql = '''
            CREATE TABLE IF NOT EXISTS repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                language TEXT,
                stars INTEGER,
                url TEXT,
                scraped_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        '''
        
        # SQL文の実行
        cur.execute(sql)
        conn.commit()
        
        print(f"データベース '{db_name}' を作成しました")
        return conn
        
    except sqlite3.Error as e:
        print('エラーが発生しました:', e)
        return None

In [17]:
def parse_star_count(stars_text):
    """スター数のテキストを数値に変換"""
    if not stars_text:
        return 0
    
    stars_text = stars_text.strip().replace(',', '')
    
    try:
        if 'k' in stars_text.lower():
            return int(float(stars_text.lower().replace('k', '')) * 1000)
        elif stars_text.replace('.', '').isdigit():
            return int(float(stars_text))
    except:
        pass
    
    return 0

def get_repo_details(repo_url):
    """個別のリポジトリページから詳細情報を取得"""
    try:
        response = requests.get(repo_url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 言語情報を取得
        language = 'N/A'
        lang_tag = soup.find('span', {'itemprop': 'programmingLanguage'})
        if lang_tag:
            language = lang_tag.text.strip()
        
        # スター数を取得
        stars = 0
        # パターン1: id="repo-stars-counter-star"
        star_tag = soup.find('span', {'id': 'repo-stars-counter-star'})
        if star_tag:
            stars_text = star_tag.get('title', star_tag.text)
            stars = parse_star_count(stars_text)
        else:
            # パターン2: stargazersへのリンク
            star_link = soup.find('a', href=re.compile(r'/stargazers$'))
            if star_link:
                stars_text = star_link.text.strip()
                stars = parse_star_count(stars_text)
        
        return language, stars
        
    except Exception as e:
        print(f"    詳細取得エラー: {e}")
        return 'N/A', 0


In [18]:
def scrape_github_repos(org_url, max_pages=5):
    """GitHubのorganizationページからリポジトリ情報をスクレイピング"""
    repositories = []
    
    print(f"\nスクレイピング開始: {org_url}")
    
    # 複数ページを取得（GitHubはページネーションがある場合）
    for page in range(1, max_pages + 1):
        url = f"{org_url}?page={page}&type=source"
        print(f"\nページ {page} を取得中...")
        
        try:
            response = requests.get(url, headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            })
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # リポジトリリストを取得
            repo_links = soup.find_all('a', href=re.compile(r'^/google/[^/]+$'))
            
            if not repo_links:
                print(f"ページ {page} にリポジトリが見つかりませんでした")
                if page == 1:
                    print("デバッグ情報:ページの一部を表示します")
                    print(soup.get_text()[:500])
                break

            print(f" {len(repo_links)} 件のリポジトリを発見:")

            #new_repos_countを初期化
            new_repos_count = 0

            #重複を避けるため、処理済みリポジトリ名を記録
            processed_repos = set(repo['name'] for repo in repositories)
            
            for link in repo_links:
                try:
                    # リポジトリ名とURLを取得
                    repo_name = link.text.strip()

                    #空白文字や重複はスキップ
                    if not repo_name or repo_name in processed_repos:
                        continue
                
                    repo_url = f"https://github.com{link['href']}"
                    print(f" {repo_name}の詳細を取得中…", end='')

                    language, stars = get_repo_details(repo_url)
                    
                    repositories.append({
                        'name': repo_name,
                        'language': language,
                        'stars': stars,
                        'url': repo_url
                    })
                    
                    processed_repos.add(repo_name)
                    print(f" ✓ ({language}) ⭐ {stars:,}")
                    new_repos_count += 1
                    
                    # アクセス集中を防ぐ（1秒待機）
                    time.sleep(1)
                    
                except Exception as e:
                    print(f"  ✗ リポジトリ情報の取得エラー: {e}")
                    continue
            
            print(f"  → {new_repos_count} 件の新規リポジトリを取得\n")
            
            # アクセス集中を防ぐ（1秒待機）
            if page < max_pages:
                time.sleep(1)
            
        except requests.RequestException as e:
            print(f"ページ取得エラー: {e}")
            break
    
    print(f"\n 合計 {len(repositories)} 件のリポジトリを取得しました")
    return repositories
    

In [19]:
def save_to_database(conn, repositories):
    #リポジトリ情報をデータベースに保存
    if conn is None:
        print("データベース接続が確立されていません。")
        return
    
    try:
        #SQL実行用のカーソルオブジェクトを作成
        cur = conn.cursor()

        print(f"\n{len(repositories)}件のリポジトリをデータベースに保存中…")

        for repo in repositories:
            #SQL文の作成，データの挿入
            sql = '''
                INSERT INTO repositories (name, language, stars, url)
                VALUES (?, ?, ?, ?);
            '''
            #SQLの実行
            cur.execute(sql, (repo['name'], repo['language'], repo['stars'], repo['url']))
        
        #変更をコミットして保存
        conn.commit()
        print(f"保存完了しました。")
    except sqlite3.Error as e:
        print(f"データベース保存エラー:{e}")

In [20]:
def display_repositories(conn):
    #データベース内のリポジトリ情報を表示
    if conn is None:
        print("データベース接続が確立されていません。")
        return  
    try:
        #SQL文の作成，データの取得
        cur = conn.cursor()

        print("\n" +"=" * 80)
        print("データベースに保存されたリポジトリ一覧")
        print("=" * 80)

        #SQL文の作成，データの取得
        sql = '''
            SELECT name, language, stars, url 
            FROM repositories
            ORDER BY stars DESC;
        '''
        #SQLの実行
        cur.execute(sql)

        #結果の取得
        results = cur.fetchall()

        if not results:
            print("データが見つかりませんでした。")
            return

        print(f"\n総リポジトリ数: {len(results)}件")

        for i, (name, language, stars, url) in enumerate(results, 1):
            print(f"{i}, {name}")
            print(f"   言語: {language}")
            print(f"   スター数: {stars}")
            print(f"   URL: {url}\n")
            print()

        #統計情報を表示
        sql_stats = '''
            SELECT language, COUNT(*) as count, SUM(stars) as total_stars
            FROM repositories
            GROUP BY language
            ORDER BY count DESC;
        '''
        cur.execute(sql_stats)
        lang_stats = cur.fetchall()

        print("="*80)
        print("プログラミンふ言語別統計")

    except sqlite3.Error as e:
        print(f"エラーが発生しました:{e}")

In [21]:
def main():
    conn = None
    
    try:
        # データベースの作成
        conn = create_database()
        
        if conn is None:
            print("データベースの作成に失敗しました")
            return
        
        # GitHubのorganizationページをスクレイピング
        org_url = "https://github.com/google"
        repositories = scrape_github_repos(org_url, max_pages=3)
        
        if repositories:
            # データベースに保存
            save_to_database(conn, repositories)
            
            # データを表示（SELECT文で取得）
            display_repositories(conn)
        else:
            print("リポジトリが取得できませんでした")
    
    except Exception as e:
        print('予期しないエラーが発生しました:', e)
    
    finally:
        # DBへの接続を閉じる
        if conn:
            conn.close()
            print("\nデータベース接続を閉じました")
        print("処理完了!")

if __name__ == "__main__":
    main()

データベース 'github_repositories.db' を作成しました

スクレイピング開始: https://github.com/google

ページ 1 を取得中...
 16 件のリポジトリを発見:
 material-design-iconsの詳細を取得中… ✓ (N/A) ⭐ 52,557
 guavaの詳細を取得中… ✓ (N/A) ⭐ 51,290
 zxの詳細を取得中… ✓ (N/A) ⭐ 44,858
 styleguideの詳細を取得中… ✓ (N/A) ⭐ 38,690
 leveldbの詳細を取得中… ✓ (N/A) ⭐ 38,361
 googletestの詳細を取得中… ✓ (N/A) ⭐ 37,497
 nomulusの詳細を取得中… ✓ (N/A) ⭐ 1,767
 meridianの詳細を取得中… ✓ (N/A) ⭐ 1,186
 device-infraの詳細を取得中… ✓ (N/A) ⭐ 58
 angleの詳細を取得中… ✓ (N/A) ⭐ 3,841
 heirの詳細を取得中… ✓ (N/A) ⭐ 608
 gn-language-serverの詳細を取得中… ✓ (N/A) ⭐ 18
 zerocopyの詳細を取得中… ✓ (N/A) ⭐ 2,075
 matrの詳細を取得中… ✓ (N/A) ⭐ 8
 docsyの詳細を取得中… ✓ (N/A) ⭐ 2,867
 dawnの詳細を取得中… ✓ (N/A) ⭐ 776
  → 16 件の新規リポジトリを取得


ページ 2 を取得中...
 16 件のリポジトリを発見:
  → 0 件の新規リポジトリを取得


ページ 3 を取得中...
 16 件のリポジトリを発見:
  → 0 件の新規リポジトリを取得


 合計 16 件のリポジトリを取得しました

16件のリポジトリをデータベースに保存中…
保存完了しました。

データベースに保存されたリポジトリ一覧

総リポジトリ数: 32件
1, material-design-icons
   言語: N/A
   スター数: 52557
   URL: https://github.com/google/material-design-icons


2, guava
   言語: N/A
   スター数: 5